In [5]:
# Import package which checks particle location
from ipynb.fs.defs.check_location import check_location

# Import gradient of flow map
from ipynb.fs.defs.gradient_flowmap import gradient_flowmap

# Import package for computing CauchyGreen (CG) strain tensor
from ipynb.fs.defs.CauchyGreen import CauchyGreen

# Import package for computing Finite-Time Lyapunov Exponent (FTLE)
from ipynb.fs.defs.FTLE import _FTLE

# import numpy
import numpy as np

In [6]:
# parallel computation of FTLE over meshgrid.
def parallel_FTLE(time, X_domain, Y_domain, X, Y, time_data, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta, aux_grid, i):
    
    FTLE_parallel = X_domain[0,:].copy()*np.nan
    
    for j in range(X_domain.shape[1]):
        
        print("i:", i)
        print("j:", j)
        
        # set initial condition
        x = np.array([X_domain[i, j], Y_domain[i, j]])
        
        print(x)
        
        # only compute FTLE for trajectories starting region where velocity field is defined
        if check_location(X, Y, defined_domain, x)[0] == "IN":
    
            # compute gradient of flow map from finite differencing 
            gradFmap = gradient_flowmap(time, x, X, Y, time_data, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta, aux_grid)
        
            # gradFmap has shape (2, 2, len(time)) --> we need gradient of flow map from t0 to tN
            gradFmap_t0_tN = gradFmap[:,:,-1]
        
            # compute CG
            C = CauchyGreen(gradFmap_t0_tN)
            
            # compute FTLE from CG
            FTLE_parallel[j] = _FTLE(C, lenT)
        
    return FTLE_parallel